In [9]:
# important imports
import numpy as np
import math
import itertools
import time
from multiprocessing import Pool
import scipy.sparse
from IPython.display import clear_output
from multiprocessing.dummy import Pool as ThreadPool
import csv
import ast
import collections
import random
import matplotlib.pyplot as plt
import networkx as nx


In [10]:
#Given an omino (nxn binary matrix), checks if it is valid
# i.e. all ones are rook-adjacent and contiguous

def valid_omino(omino):
    '''
    takes an omino as input and validates it
    returns a boolean
    '''
    
    k = omino.shape[0]
    check = np.zeros([k,k]) - omino
    
    
    _f  = False
    for i in range(k):
        for j in range(k):
            if check[i,j] == -1:
                if not _f:
                    _f  = True
                    check[i,j] = 1
    if not _f:
        print("ABORT")
        return
    
    # I know this should require fewer passes, but...
    for iter in range(k**2):
        for i in range(k):
            for j in range(k):
                if check[i,j] == 1:
                    if i != 0:
                        check[i-1,j] = check[i-1,j]**2

                    if i != k-1:
                        check[i+1,j] = check[i+1,j]**2

                    if j != 0:
                        check[i,j-1] = check[i,j-1]**2

                    if j != k-1:
                        check[i,j+1] = check[i,j+1]**2
                    
    return np.sum(check) == np.sum(omino)


# given a list of (r,c) index pairs, generates an omino
# if the omino isn't valid, returns the 0x0 0 matrix
def make_omino(inds, dim):
    '''
    takes a set of indices (x,y) to be set to 1
    makes a matrix of size dim x dim
    returns the matrix if it is a valid omino
    returns the empty zeros matrix otherwise
    
    '''
    p = np.zeros([dim,dim])
    for loc in inds:
        p[loc[0],loc[1]] = 1
    
    if valid_omino(p):
        return p
    else:
        return np.zeros([0,0])

    

# generates all of the <cells>-ominos in the <grid>^2 grid
def make_omino_set(cells, grid):
    '''
    takes two numbers, cells and grid and returns a list
    of all the valid cells-ominos which fit in the
    grid^2 grid
    
    returns a list of valid ominos    
    '''
    
    if grid % cells != 0:
        print("WARNING - <cells> doesnt divide <grid>")
        #return []
    
    
    pair_idx = []
    for i in range(grid):
        for j in range(grid):
            pair_idx.append((i,j))
    
    
    ominos = []
    
    for t in (l for l in itertools.combinations(pair_idx, cells)):
        p = make_omino(t,grid)
        if p.shape[0] != 0:
            ominos.append(p)
    
    return ominos


In [11]:
n = make_omino_set(3,4)
n += make_omino_set(4,4)
n += make_omino_set(5,4)



print(n[0])

WARNING - <cells> doesnt divide <grid>
WARNING - <cells> doesnt divide <grid>
[[1. 1. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [12]:
# since matrices don't hash, here's how this all works:
# 1. generate the set of legal ominos
# 2. make a list of indices into that lis
# 3. generate all of the tuples of indices
#    which correspond to valid pairs, triples, quads,
#    etc. of these ominos
#    a tuple is "valid" if none of them overlap
# 4. use these tuples to index back into the set
#    of ominos


# declare some variables to make later stuff work:
pairs_inds = []
trips_inds = []
quads_inds = []
inds = []

def make_trips(inx):
    '''
    takes in an index and returns the valid triples
    of ominos which include that index
    
    designed to be used with a .map(make_trips, inds) call 
    
    '''
    val_trips = []
    counter = 0
    global pairs_inds
    for d in pairs_inds:
        c=inx
        counter +=1
        #if counter%10000 == 0: print(counter/10000)
        q = set(d+(c,))
        if len(q) == 3 and c>max(d):
            check  = sum([n[i] for i in q])
            if 2 not in check: val_trips.append(tuple(q))
    #print("returning on ",inx)
    return val_trips

def make_quads(inx):
    '''
    takes in an index and returns the valid four-tuples
    of ominos which include that index
    
    designed to be used with a .map(make_quads, inds) call 
    
    '''
    val_quads = []
    counter = 0
    global trips_inds
    for t in trips_inds:
        c=inx
        counter +=1
        #if counter%10000 == 0: print(counter/10000)
        p = set(t+(c,))
        if len(p) == 4 and c>max(t):
            check  = sum([n[i] for i in p])
            if 2 not in check: val_quads.append(tuple(p))
    #if inx%10==0: print("returning on ",inx)
    return val_quads




def make_pents(inx):
    '''
    takes in an index and returns the valid five-tuples
    of ominos which include that index
    
    designed to be used with a .map(make_pents, inds) call 
    
    '''
    val_pents = []
    counter = 0
    global quads_inds
    for t in quads_inds:
        c=inx
        counter +=1
        #if counter%10000 == 0: print(counter/10000)
        p = set(t+(c,))
        if len(p) == 5 and c>max(t):
            check  = sum([n[i] for i in p])
            if 2 not in check and valid_omino(ones-check): val_pents.append(tuple(p))
    #if inx%10==0: print("returning on ",inx)
    return val_pents

In [13]:
# this box gives the partitions of the 4x4 grid into
# 4 regions of size either 3, 4, or 5

tic = time.time()

n = make_omino_set(3,4)
n += make_omino_set(4,4)
n += make_omino_set(5,4)
print(len(n))
inds = range(len(n))
pairs_inds = [x for x in itertools.combinations(inds,2) if 2 not in n[x[0]]+n[x[1]]]

# deploy some threads to make the valid triples
pool = Pool(8)
results = pool.map(make_trips,inds)
trips_inds  = [item for sublist in results for item in sublist]
trips_inds = list(set([tuple(sorted(j)) for j in trips_inds]))
pool = Pool(8)

# we can throw out a bunch of these triples:
ones = np.ones([4,4])
trips_inds = [ t for t in trips_inds if valid_omino(ones-sum([n[i] for i in t]))]

# and now the fours
results = pool.map(make_quads,inds)
quads_inds = [item for sublist in results for item in sublist]
quads_inds = list(set([tuple(sorted(j)) for j in quads_inds]))

quads_inds = [q for q in quads_inds if 0 not in sum([n[i] for i in q])]
print(len(quads_inds))

WARNING - <cells> doesnt divide <grid>
WARNING - <cells> doesnt divide <grid>
409
1953


In [14]:
def check_adjacent(q1, q2):
    if len(set(q1+q2)) != 6: return False
    
    com = set(q1).intersection(set(q2))
    
    c1 = [x for x in q1 if x not in com]
    c2 = [x for x in q2 if x not in com]
    
    t1 = n[c1[0]] - n[c2[0]]
    t2 = n[c1[0]] - n[c2[1]]
    t1=np.absolute(t1)
    t2=np.absolute(t2)
    
    if np.sum(t1)!=1 and np.sum(t2)!=1: return False
    return True

In [15]:
graph = collections.defaultdict(set)
for q1 in quads_inds:
    for q2 in quads_inds:
        if check_adjacent(q1,q2):
            graph[q1].add(q2)

In [16]:
print(len(graph[quads_inds[1]]), time.time()-tic)
print(sum([i*n[i].astype(int) for i in quads_inds[1]]))
for x in graph[quads_inds[1]]:
    print(sum([i*n[i] for i in x]))

8 98.10304498672485
[[  2   2  83  83]
 [315   2  83  83]
 [315 315 315 163]
 [315 163 163 163]]
[[ 58.  58.  12.  12.]
 [315.  58.  58.  12.]
 [315. 315. 315. 163.]
 [315. 163. 163. 163.]]
[[  2.   2. 251. 251.]
 [315.   2. 251. 251.]
 [315. 315. 315. 251.]
 [315.  51.  51.  51.]]
[[  2.   2.  83.  83.]
 [109.   2.  83.  83.]
 [109. 109. 406. 406.]
 [109. 406. 406. 406.]]
[[  2.   2.  11.  11.]
 [315.   2.  11. 384.]
 [315. 315. 315. 384.]
 [315. 384. 384. 384.]]
[[ 54.  54.  54.  16.]
 [315.  54.  16.  16.]
 [315. 315. 315. 163.]
 [315. 163. 163. 163.]]
[[  2.   2. 250. 250.]
 [109.   2. 250. 250.]
 [109. 109. 250. 163.]
 [109. 163. 163. 163.]]
[[ 56.  56.  83.  83.]
 [ 56.  56.  83.  83.]
 [145. 145. 145. 163.]
 [145. 163. 163. 163.]]
[[  2.   2.  83.  83.]
 [108.   2.  83.  83.]
 [108. 108. 108. 408.]
 [408. 408. 408. 408.]]


In [23]:

samp = []
for i in range(100):
    visited = set()
    start = random.choice(quads_inds)
    curr = start
    visited.add(curr)
    count = 0
    while len(visited) < len(quads_inds):
        curr = random.choice(list(graph[curr]))
        visited.add(curr)
        count+=1
    samp.append(count)
print("MEAN TIME TO VISIT ALL NODES: ",np.mean(samp))




G = nx.to_networkx_graph(graph)
print(nx.is_connected(G))
lap_sp = nx.laplacian_spectrum(G)
lap_sp.sort()
print("VERTICES: ", len(nx.nodes(G)))
print("EDGES: ", len(nx.edges(G)))
print("TRIANGLES: " ,sum(nx.triangles(G).values())/3)
print("DIAMETER: ", nx.diameter(G))
print("DENSITY: ", nx.density(G))
print("CLUSTERING: ", (nx.average_clustering(G)))
print("TRACE OF exp(adj): ", nx.estrada_index(G))
print("SPECTRAL GAP: ", lap_sp[1])
print("EDGE, VERTEX CONNECTIVITY: ", nx.minimum_edge_cut(G), ' ', nx.minimum_node_cut(G))




#plt.hist([len(x) for x in graph.values()], bins=15)


MEAN TIME TO VISIT ALL NODES:  37217.47
True
VERTICES:  1953
EDGES:  7024
TRIANGLES:  1084.0
DIAMETER:  12
DENSITY:  0.0036849571487329287
CLUSTERING:  0.06975681768308511
TRACE OF exp(adj):  59955.26731100199
SPECTRAL GAP:  0.41194557582335456
EDGE, VERTEX CONNECTIVITY:  {((35, 65, 138, 197), (5, 35, 197, 338)), ((35, 52, 109, 338), (5, 35, 197, 338))}   {(0, 98, 101, 404), (0, 99, 157, 298)}


In [20]:


mv = 100000
for k,v in graph.items():
    if len(v)<mv:
        mv = len(v)
        kk = k
        
print(mv, kk)


print(sum([i*n[i] for i in kk]))


for x in graph[kk]:
    print(sum([i*n[i] for i in x]))

2 (0, 17, 298, 404)
[[  0.   0.   0.  17.]
 [298. 298. 298.  17.]
 [298. 404. 298.  17.]
 [404. 404. 404. 404.]]
[[  0.   0.   0.  98.]
 [101. 101. 101.  98.]
 [101. 404.  98.  98.]
 [404. 404. 404. 404.]]
[[  0.   0.   0.  99.]
 [298. 298. 298.  99.]
 [298. 157. 298.  99.]
 [157. 157. 157.  99.]]


In [ ]:
for k,v in graph.items():
    if len(v)==16:
        paths = nx.shortest_path(G,k)

In [ ]:
path_lens = []
for k,v in paths.items():
    path_lens.append(len(v))
plt.hist(path_lens, bins=6, range=[1,6])    

In [ ]:
cent = nx.center(G)

In [ ]:
cg = G.subgraph(cent)
print(nx.is_connected(cg))
print(nx.diameter(cg))
for node in cg:
    G.remove_node(node)
    


In [ ]:
print(nx.is_connected(G))
print(nx.diameter(G))
print(len(G))


In [ ]:
paths = nx.shortest_path(G)
path_lens = []
for k,v in paths.items():
    for u in v.values():
        path_lens.append(len(u))
        
plt.hist(path_lens, bins = 9, range = [1,9])